In [231]:
# Dependencies and Setup
import pandas as pd

In [232]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


# District Summary

In [233]:
#calculate total nuber of schools
school_count = school_data_complete['school_name'].unique()


total_school_number=len(school_count)

total_school_number

15

In [234]:
# Calculate the total number of students
student_count = school_data_complete["student_name"].count()
student_count

39170

In [235]:
# Calculate the total budget
total_budget = school_data_complete["budget"].sum()
total_budget

82932329558

In [236]:
# Calculate the average math score.
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [237]:
# Calculate the average reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [238]:
# Calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [239]:
# Calculate the percentage of students with a passing reading score (70 or greater)
passing_reading_count = school_data_complete.loc[school_data_complete["reading_score"] >= 70]
number_students_passing_reading = passing_reading_count["Student ID"].count()

passing_reading_percentage = (number_students_passing_reading / total_students_number) * 100
passing_reading_percentage

85.80546336482001

In [240]:
# Calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [241]:
# Create a dataframe to hold the above results
district_summary = pd.DataFrame({
    "Total Schools": total_school_number,
    "Total Students": f"{total_students_number:,}",
    "Total Budget": f"${total_budget:,.2f}",
    "Average Math Score": f"{average_math_score:.6f}",
    "Average Reading Score": f"{average_reading_score:.5f}",
    "% Passing Math": f"{passing_math_percentage:.6f}",
    "% Passing Reading": f"{passing_reading_percentage:.6f}",
    "% Overall Passing": f"{overall_passing_rate: .6f}"
}, index=[0])

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$82,932,329,558.00",78.985371,81.87784,74.980853,85.805463,65.172326


# School Summary

In [242]:
# Determine the School Type
school_types = school_data.set_index(["school_name"])["type"]

In [243]:
# Calculate the total student count
per_school_counts = school_data_complete["school_name"].count()

In [244]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [245]:
# Calculate the average test scores
per_school_math = school_data_complete.groupby(["school_name"]).mean()["math_score"]
per_school_reading = school_data_complete.groupby(["school_name"]).mean()["reading_score"]

In [246]:
# Get the students who passed math and passed reading by creating separate filtered DataFrames.
school_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]
school_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]

# Get the the students who passed both reading and math in a separate DataFrame.
passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)
                                               & (school_data_complete["math_score"] >= 70)]

In [247]:
# percent passing overall
school_passing_math = school_passing_math.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
school_passing_reading = school_passing_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100
overall_passing_rate = passing_math_and_reading.groupby(["school_name"]).count()["student_name"] / per_school_counts * 100

In [248]:
# create data frame                                                       # refer to  GoodReadsSummary.ipynb 
per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Total School Budget": per_school_budget,
    "Per Student Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    "% Passing Math": school_passing_math,
    "% Passing Reading": school_passing_reading,
    "% Overall Passing": overall_passing_rate
})

# Minor data wrangling
per_school_summary = per_school_summary[["School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                         "Average Math Score", "Average Reading Score", 
                                         "% Passing Math", "% Passing Reading", 
                                         "% Overall Passing"]]

per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,39170,"$3,124,928.00",$79.78,77.048432,81.033963,8.470768,10.408476,6.941537
Cabrera High School,Charter,39170,"$1,081,356.00",$27.61,83.061895,83.975780,4.465152,4.603013,4.332397
Figueroa High School,District,39170,"$1,884,411.00",$48.11,76.711767,81.158020,4.968088,6.078632,4.005617
Ford High School,District,39170,"$1,763,916.00",$45.03,77.102592,80.746258,4.776615,5.545060,3.796273
Griffin High School,Charter,39170,"$917,500.00",$23.42,83.351499,83.816757,3.500128,3.640541,3.395456
Hernandez High School,District,39170,"$3,022,020.00",$77.15,77.289752,80.934412,7.898902,9.568547,6.333929
Holden High School,Charter,39170,"$248,087.00",$6.33,83.803279,83.814988,1.008425,1.049272,0.972683
Huang High School,District,39170,"$1,910,635.00",$48.78,76.629414,81.182722,4.891499,6.055655,3.985193
Johnson High School,District,39170,"$3,094,650.00",$79.01,77.072464,80.966394,8.029104,9.872351,6.507531


# Highest-Performing Schools (by % Overall Passing)

In [249]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
highest_performing_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=False)
highest_performing_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,39170,"$3,124,928.00",$79.78,77.048432,81.033963,8.470768,10.408476,6.941537
Johnson High School,District,39170,"$3,094,650.00",$79.01,77.072464,80.966394,8.029104,9.872351,6.507531
Hernandez High School,District,39170,"$3,022,020.00",$77.15,77.289752,80.934412,7.898902,9.568547,6.333929
Rodriguez High School,District,39170,"$2,547,363.00",$65.03,76.842711,80.744686,6.775594,8.189941,5.409752
Wilson High School,Charter,39170,"$1,319,574.00",$33.69,83.274201,83.989488,5.471024,5.626755,5.279551


# Lowest-Performing Schools (by % Overall Passing)

In [250]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
lowest_performing_schools = per_school_summary.sort_values(["% Overall Passing"], ascending=True)
lowest_performing_schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Holden High School,Charter,39170,"$248,087.00",$6.33,83.803279,83.814988,1.008425,1.049272,0.972683
Pena High School,Charter,39170,"$585,858.00",$14.96,83.839917,84.044699,2.323207,2.356395,2.223641
Griffin High School,Charter,39170,"$917,500.00",$23.42,83.351499,83.816757,3.500128,3.640541,3.395456
Ford High School,District,39170,"$1,763,916.00",$45.03,77.102592,80.746258,4.776615,5.545060,3.796273
Thomas High School,Charter,39170,"$1,043,130.00",$26.63,83.418349,83.848930,3.893286,4.061782,3.796273


# Math Scores by Grade

In [251]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# group by school
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()["math_score"]
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()["math_score"]
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()["math_score"]
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()["math_score"]

In [252]:
# create df
scores_by_grade = pd.DataFrame({"9th": ninth_graders_scores, "10th": tenth_graders_scores,
                                "11th": eleventh_graders_scores, "12th": twelfth_graders_scores})

# 
scores_by_grade = scores_by_grade[["9th", "10th", "11th", "12th"]]
scores_by_grade.index.name = None

# Display the DataFrame
scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


# Reading Score by Grade 

In [253]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

In [254]:
# group by school
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()["reading_score"]
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()["reading_score"]
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()["reading_score"]
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()["reading_score"]

In [255]:
# create df
scores_by_grade = pd.DataFrame({"9th": ninth_graders_scores, "10th": tenth_graders_scores,
                                "11th": eleventh_graders_scores, "12th": twelfth_graders_scores})

scores_by_grade = scores_by_grade[["9th", "10th", "11th", "12th"]]
scores_by_grade.index.name = None

# Display the DataFrame
scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


# Scores by School Spending

In [256]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [257]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [258]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = 

SyntaxError: invalid syntax (761061942.py, line 2)

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [ ]:
# Assemble into DataFrame
spending_summary = 

# Display results
spending_summary

# Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = 

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`


# Display results
size_summary

# Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = 
type_reading_scores = 
type_passing_math = 
type_passing_reading = 
type_overall_passing = 

# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`


# Display results
type_summary